In [1]:
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2
import os
import glob

# MAKE SOME SNAPSHOTS

In [2]:
# if not os.path.exists('camera_snapshots'):
#     os.mkdir('camera_snapshots')

# cap = cv2.VideoCapture(0)

# c = 0
# while True:
    
#     _, frame = cap.read()
    
#     cv2.imshow('Image', frame)
    
#     if cv2.waitKey(33) == ord('a'):
#         cv2.imwrite(f'camera_snapshots/sample_{c}.jpg', frame)
#         c += 1
    
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

# CAMERA CALIBRATION

In [3]:
import numpy as np
import cv2
import glob

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7*6,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
objp = objp * 2 

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('camera_snapshots/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

In [4]:
ret, mtx, distor, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None,  None, flags=cv2.CALIB_RATIONAL_MODEL)

ret, rvecs, tvecs = cv2.solvePnP(objp, corners2, mtx, distor)
rotMat, _ = cv2.Rodrigues(rvecs)

In [5]:
img = cv2.imread('camera_snapshots/sample_11.jpg')
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,distor,(w,h),1,(w,h))

In [7]:
dst = cv2.undistort(img, newcameramtx, distor, None, newcameramtx)

In [8]:
cv2.imshow('', dst)

cv2.waitKey(0)

-1

In [6]:
def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

def find_width():
    
#     width_const = 2.54
    
    cap = cv2.VideoCapture(0)

            
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

#     objp = np.zeros((7*7,3), np.float32)
#     objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)
#     axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

    
    
    while(True):
        
        ret, frame = cap.read()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7,7), 0)


        edged = cv2.Canny(gray, 50, 100)
        edged = cv2.dilate(edged, None, iterations=1)
        edged = cv2.erode(edged, None, iterations=1)
        
        cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        
        cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        
        
        (cnts, _) = contours.sort_contours(cnts)
        pixelsPerMetric = None

        for c in cnts:

            if cv2.contourArea(c) < 100:
                continue

            orig = frame.copy()
            box = cv2.minAreaRect(c)
            box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
            box = np.array(box, dtype="int")
            box = perspective.order_points(box)
            cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)

            for (x, y) in box:
                cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)

            (tl, tr, br, bl) = box
            (tltrX, tltrY) = midpoint(tl, tr)
            (blbrX, blbrY) = midpoint(bl, br)
         
            (tlblX, tlblY) = midpoint(tl, bl)
            (trbrX, trbrY) = midpoint(tr, br)

            cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)

            cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
                (255, 0, 255), 2)
            cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
                (255, 0, 255), 2)


            
            tempMat = np.linalg.inv(rotMat) * np.linalg.inv(newcameramtx) * [tlblX,tlblY,1]
            tempMat2 = np.linalg.inv(rotMat) * tvecs

            s = 0 + tempMat2[2][0]
            s /= tempMat[2][2]
            
            wcPoint_tlbl = np.linalg.inv(rotMat) * (s * np.linalg.inv(newcameramtx) * [tlblX, tlblY, 1] - tvecs)
            
            tempMat = np.linalg.inv(rotMat) * np.linalg.inv(newcameramtx) * [trbrX,trbrY,1]
            tempMat2 = np.linalg.inv(rotMat) * tvecs

            s = 0 + tempMat2[2][0]
            s /= tempMat[2][2]
            
            wcPoint_trbr = np.linalg.inv(rotMat) * (s * np.linalg.inv(newcameramtx) * [trbrX, trbrY, 1] - tvecs)
            
            dB = dist.euclidean((wcPoint_tlbl[0][0], wcPoint_tlbl[1][0], wcPoint_tlbl[2][0]),
                                (wcPoint_trbr[0][0], wcPoint_trbr[1][0], wcPoint_trbr[2][0]))
            
            cv2.putText(orig, "{:.1f}sm".format(dB),
                (int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,
                0.65, (255, 255, 255), 2)
            
            cv2.imshow("Image", orig)
            cv2.waitKey(0)
        

In [ ]:
find_width()